In [ ]:
import sqlite3
import os

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/falando_nela_v2/src')
    print("Current working directory (Colab):", os.getcwd())
else:
  print("Current working directory (not Colab):", os.getcwd())
DB_PATH = "../data/DiscursosSenadores.sqlite"
conn = sqlite3.connect(DB_PATH)

Mounted at /content/drive
Current working directory (Colab): /content/drive/MyDrive/falando_nela_v2/src


Carregar tabela_final.xlsx

In [12]:
import pandas as pd
import os
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')

# Caminhos dos arquivos
entrada = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_copia.xlsx'
saida = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado.xlsx'

# Carregar as planilhas
xls = pd.ExcelFile(entrada)
df_discursos = pd.read_excel(xls, sheet_name="Discursos")
df_argumentacao = pd.read_excel(xls, sheet_name="Argumentacao")
df_topicos_resumo = pd.read_excel(xls, sheet_name="TopicosResumo")
df_bm25_anual = pd.read_excel(xls, sheet_name="BM25Anual")
df_senadores = pd.read_excel(xls, sheet_name="Senadores")

# Nova planilha: Discursos
colunas_discursos = [
    'CodigoPronunciamento', 'CodigoParlamentar', 'DataPronunciamento',
    'TextoResumo', 'Indexacao', 'SiglaPartidoParlamentarNaData',
    'UfParlamentarNaData', 'SiglaCasaPronunciamento', 'Forma', 'TextoIntegral'
]
discursos_limpo = df_discursos[colunas_discursos]

# Nova planilha: AnaliseCorpusTodo
colunas_analise_corpus = [
    'CodigoPronunciamento', 'BM25_CorpusTodo_Constituição', 'TFIDF_CorpusTodo_Constituição',
    'SentimentoGeral', 'SentimentoConstituicao', 'SumarioConstituicao', 'TrechosConstituicao',
    'NovaConstituinteOuConstituicao_resposta', 'NovaConstituinteOuConstituicao_trecho',
    'TopicosConstituicao', 'MencionaConstituicao',
    'NormPredicacao', 'NormImplicacao', 'NormConclusao', 'NormTrecho',
    'AvalPredicacao', 'AvalImplicacao', 'AvalConclusao', 'AvalTrecho',
    'NormPredicacao_topicos', 'NormImplicacao_topicos', 'NormConclusao_topicos',
    'AvalPredicacao_topicos', 'AvalImplicacao_topicos', 'AvalConclusao_topicos'
]
analise_corpus = df_discursos[colunas_analise_corpus].merge(df_topicos_resumo, on="CodigoPronunciamento", how="left")

# Nova planilha: PesquisaSemantica
colunas_pesquisa_semantica = [
    'CodigoPronunciamento',
    'Similaridade_base_democracia', 'Similaridade_ultrapassada',
    'Similaridade_prejudica_economia', 'Similaridade_direitos_demais',
    'Similaridade_nova_constituinte', 'Similaridade_ffaa_poder_moderador',
    'Similaridade_voltar_ditadura', 'Similaridade_governo_nao_respeita',
    'Similaridade_camara_nao_respeita', 'Similaridade_supremo_nao_respeita',
    'Similaridade_ninguem_respeita'
]
pesquisa_semantica = df_discursos[colunas_pesquisa_semantica]

# Nova planilha: AnaliseAnual (Argumentacao + BM25)
analise_anual = df_argumentacao.merge(df_bm25_anual, on="CodigoPronunciamento", how="left")

# Salvar novo arquivo reorganizado
with pd.ExcelWriter(saida, engine='xlsxwriter') as writer:
    discursos_limpo.to_excel(writer, sheet_name="Discursos", index=False)
    analise_corpus.to_excel(writer, sheet_name="AnaliseCorpusTodo", index=False)
    pesquisa_semantica.to_excel(writer, sheet_name="PesquisaSemantica", index=False)
    analise_anual.to_excel(writer, sheet_name="AnaliseAnual", index=False)
    df_senadores.to_excel(writer, sheet_name="Senadores", index=False)

print(f"✅ Arquivo reorganizado salvo em: {saida}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Arquivo reorganizado salvo em: /content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado.xlsx


In [13]:
import pandas as pd
from IPython.display import display
import random

# Caminho do arquivo reorganizado
arquivo = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado.xlsx'

# Carregar todas as planilhas
planilhas = pd.read_excel(arquivo, sheet_name=None)

# Selecionar 10 discursos aleatórios
df_discursos = planilhas["Discursos"]
amostra = df_discursos.sample(n=10, random_state=42)
codigos_amostra = amostra["CodigoPronunciamento"].tolist()

# Exibir os dados de cada planilha com base nos códigos selecionados
print("🗂️ AMOSTRA: Planilha 'Discursos'")
display(amostra)

for nome, df in planilhas.items():
    if nome == "Discursos":
        continue  # já mostramos
    print(f"\n🧩 Planilha '{nome}':")
    if "CodigoPronunciamento" in df.columns:
        df_filtrado = df[df["CodigoPronunciamento"].isin(codigos_amostra)]
    elif "CodigoParlamentar" in df.columns and "CodigoParlamentar" in amostra.columns:
        codigos_parlamentares = amostra["CodigoParlamentar"].unique()
        df_filtrado = df[df["CodigoParlamentar"].isin(codigos_parlamentares)]
    else:
        df_filtrado = df  # caso especial
    display(df_filtrado)


🗂️ AMOSTRA: Planilha 'Discursos'


,CodigoPronunciamento,CodigoParlamentar,DataPronunciamento,TextoResumo,Indexacao,SiglaPartidoParlamentarNaData,UfParlamentarNaData,SiglaCasaPronunciamento,Forma,TextoIntegral
66730,509400,5793,2024-10-08,Congratulações ao Sr. Gabriel Galípolo pela a...,"CONGRATULAÇÕES, Gabriel Galípolo, APROVAÇÃO, I...",NaN,NaN,SF,texto,"O SR. DR. HIRAN – Presidente Rodrigo, Sras. e ..."
29890,401112,643,2013-07-16,Registro da apresentação de requerimento solic...,"REGISTRO, APRESENTAÇÃO, REQUERIMENTO, AUTORIA,...",NaN,NaN,SF,texto,A SRª LÚCIA VÂNIA (Bloco/PSDB - GO...
45801,445966,5533,2018-05-29,Comentários acerca da greve dos caminhoneiros ...,"COMENTARIO, SITUAÇÃO, CRISE, ECONOMIA NACIONAL...",NaN,NaN,SF,texto,O SR. LASIER MARTINS (Bloco Parlamentar De...
29352,400519,4575,2013-06-18,Referente ao PLS n. 240/2013-Complementar.,NaN,NaN,NaN,SF,texto,A SRª LÍDICE DA MATA (Bloco/PSB - ...
19858,389390,4539,2011-08-16,Críticas ao veto da Presidente Dilma Rousseff ...,"COMENTARIO, ARTIGO DE IMPRENSA, JORNAL, O GLOB...",NaN,NaN,SF,texto,O SR. MÁRIO COUTO (Bloco/PSDB - PA...
15364,384266,945,2010-05-05,"Registro do artigo intitulado ""Opção preferenc...","SOLICITAÇÃO, TRANSCRIÇÃO, ANAIS DO SENADO, ART...",NaN,NaN,SF,texto,SENADO FEDERAL SF -\...
30026,401255,846,2013-06-26,Referente ao PLS n. 204/2011.,NaN,NaN,NaN,SF,texto,O SR. ALOYSIO NUNES FERREIRA (Bloc...
38039,418039,456,2015-10-27,Sessão de Debates Temáticos destinada a debate...,"CRITICA, ALEGAÇÕES, GOVERNO FEDERAL, ALTERAÇÃO...",NaN,NaN,SF,texto,O SR. RONALDO CAIADO (Bloco Oposição/DEM -...
14261,383053,28,2010-02-12,Considerações sobre projeto de autoria de S.Ex...,"COMENTARIO, ARTIGO DE IMPRENSA, JORNAL, FOLHA ...",NaN,NaN,SF,texto,SENADO FEDERAL SF -\...
11661,380210,3364,2009-07-09,Defesa da instalação da CPI da Petrobrás. Crít...,"ANALISE, CONDUTA, PRESIDENTE DA REPUBLICA, QUE...",NaN,NaN,SF,texto,O SR. MÃO SANTA (PMDB - PI. Pronun...



🧩 Planilha 'AnaliseCorpusTodo':


,CodigoPronunciamento,BM25_CorpusTodo_Constituição,TFIDF_CorpusTodo_Constituição,SentimentoGeral,SentimentoConstituicao,SumarioConstituicao,TrechosConstituicao,NovaConstituinteOuConstituicao_resposta,NovaConstituinteOuConstituicao_trecho,TopicosConstituicao,...,AvalImplicacao,AvalConclusao,AvalTrecho,NormPredicacao_topicos,NormImplicacao_topicos,NormConclusao_topicos,AvalPredicacao_topicos,AvalImplicacao_topicos,AvalConclusao_topicos,TopicoResumo
11661,380210,0.936346,0.008278,Negativo,Neutro,Não se aplica,NaN,Não se aplica,Não se aplica,10_Não se aplica,...,"isso significa que, embora a Constituição tenh...","por causa disso, o orador sugere que o poder p...",A Constituinte não teve. Ela foi bem feita. O ...,['Constituição e direitos fundamentais no Bras...,['responsabilidade e implicações constituciona...,['Importância do Senado na democracia e no pac...,['Constituição e proteção de direitos fundamen...,['insuficiências da legislação e estrutura con...,['tópico: harmonia entre os Poderes e respeito...,"[""['Critica o Governo Federal sobre a instalaç..."
14261,383053,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15364,384266,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19858,389390,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29352,400519,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29890,401112,2.159502,0.036015,Negativo,Neutro,Não se aplica,NaN,Não se aplica,Não se aplica,1_Não se aplica,...,NaN,NaN,NaN,['Convocação de Ministros e Solicitação de Inf...,['Fiscalização do Poder Executivo pelo Senado'],['tópico: instalação de CPIs para investigar c...,['Indeterminado'],['Indeterminado'],['Nenhum tópico identificado'],"[""['Participação e relatos de audiências públi..."
30026,401255,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38039,418039,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45801,445966,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66730,509400,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



🧩 Planilha 'PesquisaSemantica':


,CodigoPronunciamento,Similaridade_base_democracia,Similaridade_ultrapassada,Similaridade_prejudica_economia,Similaridade_direitos_demais,Similaridade_nova_constituinte,Similaridade_ffaa_poder_moderador,Similaridade_voltar_ditadura,Similaridade_governo_nao_respeita,Similaridade_camara_nao_respeita,Similaridade_supremo_nao_respeita,Similaridade_ninguem_respeita
11661,380210,0.205430,0.253313,0.220365,0.244089,0.209207,0.210309,0.222963,0.291622,0.278932,0.287107,0.283312
14261,383053,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
15364,384266,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
19858,389390,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
29352,400519,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
29890,401112,0.205411,0.253272,0.220384,0.244043,0.209161,0.210362,0.222935,0.291621,0.278936,0.287127,0.283293
30026,401255,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
38039,418039,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
45801,445966,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
66730,509400,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366



🧩 Planilha 'AnaliseAnual':


,Trecho_Anual,Topic,Topico_Anual,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,CodigoPronunciamento,Ano,Tipo_Anual,BM25_anual
12174,"por causa disso, o orador sugere que o poder p...",-1,-1_['Garantia de aplicação correta e rigorosa ...,"[""['Garantia de aplicação correta e rigorosa d...","['por causa disso, o orador apela para que o S...",['Garantia de aplicação correta e rigorosa das...,0.000000,False,380210,2009,AvalConclusao,1.122038
12175,"isso significa que, embora a Constituição tenh...",-1,-1_['Garantia e preservação dos direitos funda...,"[""['Garantia e preservação dos direitos fundam...",['isso significa que as decisões do Congresso ...,['Garantia e preservação dos direitos fundamen...,0.000000,False,380210,2009,AvalImplicacao,1.122038
12176,"a Constituição foi bem feita, mas com limitaçõ...",35,35_['Apreciação das qualidades e sensibilidade...,"[""['Apreciação das qualidades e sensibilidade ...","['a Constituição é um documento que, embora te...",['Apreciação das qualidades e sensibilidade da...,1.000000,False,380210,2009,AvalPredicacao,1.122038
12177,"por causa disso, o orador defende a importânci...",-1,-1_['Criticar e reformar políticas governament...,"[""['Criticar e reformar políticas governamenta...","['por causa disso, o orador defende que, se ne...",['Criticar e reformar políticas governamentais...,0.000000,False,380210,2009,NormConclusao,1.122038
12178,isso implica que o poder não deve ser absoluto...,-1,-1_['Cumprimento das funções constitucionais e...,"[""['Cumprimento das funções constitucionais e ...",['isso implica que o Tribunal de Contas deve e...,['Cumprimento das funções constitucionais e de...,0.000000,False,380210,2009,NormImplicacao,1.122038
12179,a Constituição estabelece a divisão dos podere...,9,"9_['Separação, independência e harmonia entre ...","[""['Separação, independência e harmonia entre ...",['a Constituição estabelece a separação dos po...,"['Separação, independência e harmonia entre os...",0.589692,False,380210,2009,NormPredicacao,1.122038
32040,NaN,0,0_['Conclusões políticas ou normativas não ide...,"[""['Conclusões políticas ou normativas não ide...","['None', 'None', 'None']",['Conclusões políticas ou normativas não ident...,1.000000,True,401112,2013,AvalConclusao,2.054893
32041,NaN,0,0_['Desafios na definição de políticas pública...,"[""['Desafios na definição de políticas pública...","['None', 'None', 'None']",['Desafios na definição de políticas públicas ...,0.690757,True,401112,2013,AvalImplicacao,2.054893
32042,NaN,0,0_Discussão ambígua e indefinida sobre a Const...,['Discussão ambígua e indefinida sobre a Const...,"['None', 'None', 'None']",Discussão ambígua e indefinida sobre a Constit...,0.839776,True,401112,2013,AvalPredicacao,2.054893
32043,"por causa disso, a oradora solicita a convocaç...",-1,-1_['Revisão e atualização das leis trabalhist...,"[""['Revisão e atualização das leis trabalhista...","['por causa disso, o orador defende que o proj...",['Revisão e atualização das leis trabalhistas ...,0.000000,False,401112,2013,NormConclusao,2.054893



🧩 Planilha 'Senadores':


,CodigoParlamentar,NomeParlamentar,SexoParlamentar,NomeProfissao,IndicadorAtividadePrincipal,NomeCargo,DataInicioCargo,DataFimCargo,Orgao,NomeCurso,UnidadeLideranca,DescricaoTipoLideranca,DataInicioLideranca,DataFimLideranca
2309,28,Gerson Camata,Masculino,NaN,NaN,2º SECRETÁRIO,2007-02-01,2009-02-01,MESA DO SENADO FEDERAL,Ciências Econômicas,Liderança de Bloco no Senado Federal,Vice-líder do Senado Federal,2009-03-02,2010-12-31
2310,28,Gerson Camata,Masculino,NaN,NaN,2º SECRETÁRIO,2007-02-01,2009-02-01,MESA DO SENADO FEDERAL,Ciências Econômicas,Liderança de Partido no Senado Federal,Vice-líder do Senado Federal,1995-02-24,1999-01-31
2311,28,Gerson Camata,Masculino,NaN,NaN,2º SECRETÁRIO,2007-02-01,2009-02-01,MESA DO SENADO FEDERAL,Ciências Econômicas,Liderança de Partido no Senado Federal,Vice-líder do Senado Federal,2005-02-18,2006-03-14
2312,28,Gerson Camata,Masculino,NaN,NaN,2º SECRETÁRIO,2007-02-01,2009-02-01,MESA DO SENADO FEDERAL,Ciências Econômicas,Liderança de Partido no Senado Federal,Vice-líder do Senado Federal,2006-03-15,2006-05-03
2313,28,Gerson Camata,Masculino,NaN,NaN,4º SUPLENTE,2009-02-04,2011-02-01,MESA DO SENADO FEDERAL,Ciências Econômicas,Liderança de Bloco no Senado Federal,Vice-líder do Senado Federal,2009-03-02,2010-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27452,5793,Dr. Hiran,Masculino,NaN,NaN,RELATOR,2023-02-15,NaN,Comissão Temporária Externa para acompanhar a ...,NaN,Liderança de Partido na CMO,Líder do Congresso Nacional,2024-05-06,NaN
27453,5793,Dr. Hiran,Masculino,NaN,NaN,RELATOR,2023-02-15,NaN,Comissão Temporária Externa para acompanhar a ...,NaN,Liderança do Governo no Congresso Nacional,Vice-líder do Congresso Nacional,2021-04-28,2023-01-31
27454,5793,Dr. Hiran,Masculino,NaN,NaN,VICE-PRESIDENTE,2024-04-24,NaN,CPI DA BRASKEM,NaN,Liderança de Partido na CMO,Líder do Congresso Nacional,2023-04-13,2024-03-31
27455,5793,Dr. Hiran,Masculino,NaN,NaN,VICE-PRESIDENTE,2024-04-24,NaN,CPI DA BRASKEM,NaN,Liderança de Partido na CMO,Líder do Congresso Nacional,2024-05-06,NaN


In [16]:
import pandas as pd
from IPython.display import display
import random

# Aumenta os limites de exibição
pd.set_option('display.max_colwidth', None)  # mostra todo o conteúdo da célula
pd.set_option('display.max_columns', None)   # mostra todas as colunas
pd.set_option('display.max_rows', 100)       # ajustável conforme necessário

# Caminho do arquivo reorganizado
arquivo = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado.xlsx'

# Carregar todas as planilhas
planilhas = pd.read_excel(arquivo, sheet_name=None)

# Selecionar 10 discursos aleatórios
df_discursos = planilhas["Discursos"]
amostra = df_discursos.sample(n=10, random_state=42)
codigos_amostra = amostra["CodigoPronunciamento"].tolist()

# Exibir os dados da amostra da planilha Discursos
print("🗂️ AMOSTRA: Planilha 'Discursos'")
display(amostra)

# Exibir os dados correspondentes nas demais planilhas
for nome, df in planilhas.items():
    if nome == "Discursos":
        continue  # já mostramos
    print(f"\n🧩 Planilha '{nome}':")
    if "CodigoPronunciamento" in df.columns:
        df_filtrado = df[df["CodigoPronunciamento"].isin(codigos_amostra)]
    elif "CodigoParlamentar" in df.columns and "CodigoParlamentar" in amostra.columns:
        codigos_parlamentares = amostra["CodigoParlamentar"].unique()
        df_filtrado = df[df["CodigoParlamentar"].isin(codigos_parlamentares)]
    else:
        df_filtrado = df  # caso especial
    display(df_filtrado)


🗂️ AMOSTRA: Planilha 'Discursos'


CodigoPronunciamento  CodigoParlamentar DataPronunciamento  \
66730                509400               5793         2024-10-08   
29890                401112                643         2013-07-16   
45801                445966               5533         2018-05-29   
29352                400519               4575         2013-06-18   
19858                389390               4539         2011-08-16   
15364                384266                945         2010-05-05   
30026                401255                846         2013-06-26   
38039                418039                456         2015-10-27   
14261                383053                 28         2010-02-12   
11661                380210               3364         2009-07-09   

                                                                                                                                                                                                                                                                                                                   TextoResumo  \
66730                                                                                                                                                                                                 Congratulações ao Sr.  Gabriel Galípolo pela aprovação de seu nome para ocupar a presidência do Banco Central do Brasil.   
29890                                                                                                                       Registro da apresentação de requerimento solicitando a presença da Ministra  Tereza Campello , do Desenvolvimento Social e Combate à Fome, à audiência públivs na Comissão de Assuntos Sociais. \n   
45801                                                                                                                                                   Comentários acerca da greve dos caminhoneiros e relato da participação de S. Exª na Comissão Geral do Congresso destinada a discutir a alta no preço dos combustíveis.   
29352                                                                                                                                                                                                                                                                               Referente ao PLS n. 240/2013-Complementar.   
19858                                                                                                                               Críticas ao veto da Presidente Dilma Rousseff sobre a emenda ao Orçamento que garantia o aumento real, superior à inflação de 2012, para os aposentados; e outros assuntos. (como Líder)\n   
15364  Registro do artigo intitulado "Opção preferencial por ditaduras", publicado na Revista Veja, edição de 10 de março do corrente. Registro da matéria intitulada "Promotor estuda oferecer delação premiada a ex-diretores da cooperativa", publicada no jornal O Estado de S.Paulo, edição de 10 de março do corrente.\n   
30026                                                                                                                                                                                                                                                                                            Referente ao PLS n. 204/2011.   
38039                                                                                                                                                                                                                                 Sessão de Debates Temáticos destinada a debater o Regime Tributário do Simples Nacional.   
14261                                                                                                                                                                                    Considerações sobre projeto de autoria de S.Exa. que libera a comercialização de veículos de passeio movidos a diesel no Brasil. \n\t   
11661                      


🧩 Planilha 'AnaliseCorpusTodo':


,CodigoPronunciamento,BM25_CorpusTodo_Constituição,TFIDF_CorpusTodo_Constituição,SentimentoGeral,SentimentoConstituicao,SumarioConstituicao,TrechosConstituicao,NovaConstituinteOuConstituicao_resposta,NovaConstituinteOuConstituicao_trecho,TopicosConstituicao,MencionaConstituicao,NormPredicacao,NormImplicacao,NormConclusao,NormTrecho,AvalPredicacao,AvalImplicacao,AvalConclusao,AvalTrecho,NormPredicacao_topicos,NormImplicacao_topicos,NormConclusao_topicos,AvalPredicacao_topicos,AvalImplicacao_topicos,AvalConclusao_topicos,TopicoResumo
11661,380210,0.936346,0.008278,Negativo,Neutro,Não se aplica,NaN,Não se aplica,Não se aplica,10_Não se aplica,1.0,a Constituição estabelece a divisão dos poderes e os direitos constitucionais adquiridos dos funcionários públicos.,isso implica que o poder não deve ser absoluto e que há proteção legal para os funcionários públicos contra demissões arbitrárias.,"por causa disso, o orador defende a importância do Senado como um contrapoder e a necessidade de respeitar os direitos constitucionais dos funcionários.","A Constituição me garantia. O funcionário público, primeiro, tem que ter uma advertência oral; depois, tem que ter uma escrita; uma suspensão para se tirar. “Bota para fora!” é ignorância. Nós não podemos.","a Constituição foi bem feita, mas com limitações temporais.","isso significa que, embora a Constituição tenha sido elaborada com boas intenções, ela não previu adequadamente a duração dos mandatos presidenciais.","por causa disso, o orador sugere que o poder presidencial pode se tornar excessivo se não houver um equilíbrio adequado entre os poderes.",A Constituinte não teve. Ela foi bem feita. O Alvaro Dias foi constituinte. Mas eles fizeram para quatro anos.,['Constituição e direitos fundamentais no Brasil'],['responsabilidade e implicações constitucionais no governo e legislações'],['Importância do Senado na democracia e no pacto federativo do Brasil'],['Constituição e proteção de direitos fundamentais no Estado Democrático de Direito'],['insuficiências da legislação e estrutura constitucional atuais em atender necessidades sociais e de segurança'],['tópico: harmonia entre os Poderes e respeito à Constituição'],"[""['Critica o Governo Federal sobre a instalação da CPI da Petrobras e a atuação do Presidente']""]"
14261,383053,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15364,384266,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19858,389390,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29352,400519,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29890,401112,2.159502,0.036015,Negativo,Neutro,Não se aplica,NaN,Não se aplica,Não se aplica,1_Não se aplica,1.0,a Constituição permite a convocação de autoridades para prestar esclarecimentos ao Senado.,isso implica que o Senado tem o poder de fiscalizar e exigir explicações de membros do Executivo.,"por causa disso, a oradora solicita a convocação da Ministra do Desenvolvimento Social para esclarecer o episódio do Bolsa Família.","nos termos do art. 58, §2º, inciso III, da Constituição Federal, combinado com o inciso III, do art. 90, do Regimento Interno do Senado Federal, que seja convocada a Srª Tereza Campello, Ministra do Desenvolvimento Social e Combate à Fome, para, em audiência pública na Comissão de Assuntos Sociais, prestar os devidos esclarecimentos.",NaN,NaN,NaN,NaN,['Convocação de Ministros e Solicitação de Informações pelo Congresso e Senado'],['Fiscalização do Poder Executivo pelo Senado'],['tópico: instalação de CPIs para investigar corrupção e irregularidades na Petrobras'],['Indeterminado'],['Indeterminado'],['Nenhum tópico identificado'],"[""['Participação e relatos de audiências públicas e videoconferências no Senado']""]"
30026,401255,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,Na


🧩 Planilha 'PesquisaSemantica':


,CodigoPronunciamento,Similaridade_base_democracia,Similaridade_ultrapassada,Similaridade_prejudica_economia,Similaridade_direitos_demais,Similaridade_nova_constituinte,Similaridade_ffaa_poder_moderador,Similaridade_voltar_ditadura,Similaridade_governo_nao_respeita,Similaridade_camara_nao_respeita,Similaridade_supremo_nao_respeita,Similaridade_ninguem_respeita
11661,380210,0.205430,0.253313,0.220365,0.244089,0.209207,0.210309,0.222963,0.291622,0.278932,0.287107,0.283312
14261,383053,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
15364,384266,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
19858,389390,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
29352,400519,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
29890,401112,0.205411,0.253272,0.220384,0.244043,0.209161,0.210362,0.222935,0.291621,0.278936,0.287127,0.283293
30026,401255,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
38039,418039,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
45801,445966,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366
66730,509400,0.062467,0.037343,0.013557,-0.000033,0.079220,0.021263,0.075547,0.059078,0.053512,0.067522,0.060366



🧩 Planilha 'AnaliseAnual':


,Trecho_Anual,Topic,Topico_Anual,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,CodigoPronunciamento,Ano,Tipo_Anual,BM25_anual
12174,"por causa disso, o orador sugere que o poder presidencial pode se tornar excessivo se não houver um equilíbrio adequado entre os poderes.",-1,-1_['Garantia de aplicação correta e rigorosa das normas constitucionais e decisões do Supremo Tribunal Federal'],"[""['Garantia de aplicação correta e rigorosa das normas constitucionais e decisões do Supremo Tribunal Federal']""]","['por causa disso, o orador apela para que o Supremo Tribunal Federal e outros órgãos intervenham para restaurar a segurança jurídica.', 'Por causa disso, o orador defende a aplicação rigorosa das normas constitucionais para proteger os trabalhadores.', 'Por causa disso, o orador critica qualquer tentativa do Senado de não cumprir as decisões do Supremo Tribunal Federal, enfatizando a necessidade de respeitar a autoridade do tribunal.']",['Garantia de aplicação correta e rigorosa das normas constitucionais e decisões do Supremo Tribunal Federal'],0.000000,False,380210,2009,AvalConclusao,1.122038
12175,"isso significa que, embora a Constituição tenha sido elaborada com boas intenções, ela não previu adequadamente a duração dos mandatos presidenciais.",-1,-1_['Garantia e preservação dos direitos fundamentais no cumprimento dos princípios constitucionais'],"[""['Garantia e preservação dos direitos fundamentais no cumprimento dos princípios constitucionais']""]","['isso significa que as decisões do Congresso Nacional, como a aprovação da Emenda Constitucional nº 58, devem ser presumidas constitucionais.', 'isso significa que o cumprimento das normas constitucionais é essencial para o funcionamento adequado do Estado e para a justiça social.', 'Isso significa que ela deve ser defendida contra qualquer tentativa de violação dos direitos fundamentais.']",['Garantia e preservação dos direitos fundamentais no cumprimento dos princípios constitucionais'],0.000000,False,380210,2009,AvalImplicacao,1.122038
12176,"a Constituição foi bem feita, mas com limitações temporais.",35,35_['Apreciação das qualidades e sensibilidade da Constituição como reflexo dos clamores sociais'],"[""['Apreciação das qualidades e sensibilidade da Constituição como reflexo dos clamores sociais']""]","['a Constituição é um documento que, embora tenha suas falhas, foi criado por pessoas sensíveis ao povo.', 'a Constituição foi feita por figuras respeitáveis e sensíveis ao povo.', 'a Constituição foi feita ao longo do tempo e chega a um ponto tão bom como este, em que o Brasil está em condições ótimas.']",['Apreciação das qualidades e sensibilidade da Constituição como reflexo dos clamores sociais'],1.000000,False,380210,2009,AvalPredicacao,1.122038
12177,"por causa disso, o orador defende a importância do Senado como um contrapoder e a necessidade de respeitar os direitos constitucionais dos funcionários.",-1,-1_['Criticar e reformar políticas governamentais e constitucionais para garantir direitos e proteção ambiental'],"[""['Criticar e reformar políticas governamentais e constitucionais para garantir direitos e proteção ambiental']""]","['por causa disso, o orador defende que, se necessário, o caso seja levado ao Supremo Tribunal Federal para garantir esse direito.', 'Por causa disso, o orador apoia a Proposta de Emenda Constitucional nº 51, que visa proteger o cerrado e a caatinga.', 'por causa disso, o orador se opõe à reforma tributária proposta, que ele acredita que prejudicará a seguridade social.']",['Criticar e reformar políticas governamentais e constitucionais para garantir direitos e proteção ambiental'],0.000000,False,380210,2009,NormConclusao,1.122038
12178,isso implica que o poder não deve ser absoluto e que há proteção legal para os funcionários públicos contra demissões arbitrárias.,-1,-1_['Cumprimento das funções constitucionais e de fiscalização pelas instituições governamentais'],"[""['Cumprimento das funções 


🧩 Planilha 'Senadores':


,CodigoParlamentar,NomeParlamentar,SexoParlamentar,NomeProfissao,IndicadorAtividadePrincipal,NomeCargo,DataInicioCargo,DataFimCargo,Orgao,NomeCurso,UnidadeLideranca,DescricaoTipoLideranca,DataInicioLideranca,DataFimLideranca
2309,28,Gerson Camata,Masculino,NaN,NaN,2º SECRETÁRIO,2007-02-01,2009-02-01,MESA DO SENADO FEDERAL,Ciências Econômicas,Liderança de Bloco no Senado Federal,Vice-líder do Senado Federal,2009-03-02,2010-12-31
2310,28,Gerson Camata,Masculino,NaN,NaN,2º SECRETÁRIO,2007-02-01,2009-02-01,MESA DO SENADO FEDERAL,Ciências Econômicas,Liderança de Partido no Senado Federal,Vice-líder do Senado Federal,1995-02-24,1999-01-31
2311,28,Gerson Camata,Masculino,NaN,NaN,2º SECRETÁRIO,2007-02-01,2009-02-01,MESA DO SENADO FEDERAL,Ciências Econômicas,Liderança de Partido no Senado Federal,Vice-líder do Senado Federal,2005-02-18,2006-03-14
2312,28,Gerson Camata,Masculino,NaN,NaN,2º SECRETÁRIO,2007-02-01,2009-02-01,MESA DO SENADO FEDERAL,Ciências Econômicas,Liderança de Partido no Senado Federal,Vice-líder do Senado Federal,2006-03-15,2006-05-03
2313,28,Gerson Camata,Masculino,NaN,NaN,4º SUPLENTE,2009-02-04,2011-02-01,MESA DO SENADO FEDERAL,Ciências Econômicas,Liderança de Bloco no Senado Federal,Vice-líder do Senado Federal,2009-03-02,2010-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27452,5793,Dr. Hiran,Masculino,NaN,NaN,RELATOR,2023-02-15,NaN,Comissão Temporária Externa para acompanhar a situação dos Yanomami e a saída dos garimpeiros,NaN,Liderança de Partido na CMO,Líder do Congresso Nacional,2024-05-06,NaN
27453,5793,Dr. Hiran,Masculino,NaN,NaN,RELATOR,2023-02-15,NaN,Comissão Temporária Externa para acompanhar a situação dos Yanomami e a saída dos garimpeiros,NaN,Liderança do Governo no Congresso Nacional,Vice-líder do Congresso Nacional,2021-04-28,2023-01-31
27454,5793,Dr. Hiran,Masculino,NaN,NaN,VICE-PRESIDENTE,2024-04-24,NaN,CPI DA BRASKEM,NaN,Liderança de Partido na CMO,Líder do Congresso Nacional,2023-04-13,2024-03-31
27455,5793,Dr. Hiran,Masculino,NaN,NaN,VICE-PRESIDENTE,2024-04-24,NaN,CPI DA BRASKEM,NaN,Liderança de Partido na CMO,Líder do Congresso Nacional,2024-05-06,NaN


Incorporar resultado (os csvs) da nova modelagem das Conclusoes (caderno 12). Não esquecer que, no caso anual, a análise da argumentação segue formato long, com AvalPredicacao etc espraiadas em linhas diferentes.

In [17]:
import pandas as pd
import json

# Caminhos dos arquivos
arquivo_excel = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado.xlsx'
arquivo_json = '/content/drive/MyDrive/falando_nela_v2/src/complementacao_llm_2_normalized.json'

# Carregar o JSON como DataFrame
with open(arquivo_json, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

df_json = pd.DataFrame(json_data)

# Verifica se as colunas esperadas estão presentes
assert {'CodigoPronunciamento', 'NormConclusao', 'AvalConclusao'}.issubset(df_json.columns), \
    "O JSON precisa conter 'CodigoPronunciamento', 'NormConclusao' e 'AvalConclusao'"

# Carregar todas as planilhas do Excel
xls = pd.read_excel(arquivo_excel, sheet_name=None)
df_analise = xls["AnaliseCorpusTodo"]

# Substituir os valores com base na chave
df_analise_atualizado = df_analise.merge(
    df_json[['CodigoPronunciamento', 'NormConclusao', 'AvalConclusao']],
    on='CodigoPronunciamento',
    suffixes=('', '_novo'),
    how='left'
)

# Atualiza apenas os valores onde houver correspondência
df_analise_atualizado['NormConclusao'] = df_analise_atualizado['NormConclusao_novo'].combine_first(df_analise_atualizado['NormConclusao'])
df_analise_atualizado['AvalConclusao'] = df_analise_atualizado['AvalConclusao_novo'].combine_first(df_analise_atualizado['AvalConclusao'])

# Remove as colunas intermediárias
df_analise_atualizado.drop(columns=['NormConclusao_novo', 'AvalConclusao_novo'], inplace=True)

# Atualizar o dicionário de planilhas e salvar
xls['AnaliseCorpusTodo'] = df_analise_atualizado
novo_caminho = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado_atualizado.xlsx'

with pd.ExcelWriter(novo_caminho, engine='xlsxwriter') as writer:
    for nome_planilha, df in xls.items():
        df.to_excel(writer, sheet_name=nome_planilha, index=False)

print(f"✅ Substituições feitas e arquivo salvo em:\n{novo_caminho}")


✅ Substituições feitas e arquivo salvo em:
/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado_atualizado.xlsx


In [18]:
import pandas as pd
import os
from glob import glob

# Caminhos
arquivo_excel = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado_atualizado.xlsx'
pasta_csvs = '/content/drive/MyDrive/falando_nela_v2/data/topicos_gerados/v_12'

# Carrega todas as planilhas do Excel
planilhas = pd.read_excel(arquivo_excel, sheet_name=None)

# Localiza todos os arquivos CSV no padrão desejado
csvs = glob(os.path.join(pasta_csvs, '*_topicos_corpus_todo.csv'))

# DataFrame acumulador
df_final = None

# Itera sobre os CSVs e faz o merge incremental
for caminho_csv in csvs:
    nome_arquivo = os.path.basename(caminho_csv)
    prefixo = nome_arquivo.replace('_topicos_corpus_todo.csv', '')

    df_csv = pd.read_csv(caminho_csv)

    if 'CodigoPronunciamento' not in df_csv.columns:
        print(f"⚠️ Pulando {nome_arquivo}: sem coluna 'CodigoPronunciamento'")
        continue

    # Renomeia todas as colunas, exceto CodigoPronunciamento
    colunas_renomeadas = {
        col: f"{prefixo}_{col}" for col in df_csv.columns if col != 'CodigoPronunciamento'
    }
    df_csv = df_csv.rename(columns=colunas_renomeadas)

    # Merge incremental por CodigoPronunciamento
    if df_final is None:
        df_final = df_csv
    else:
        df_final = pd.merge(df_final, df_csv, on='CodigoPronunciamento', how='outer')

# Ordena resultado final
df_final = df_final.sort_values("CodigoPronunciamento")

# Adiciona como nova aba
planilhas["TopicosCorpusTodo"] = df_final

# Salva novo arquivo
saida = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado_topicos.xlsx'
with pd.ExcelWriter(saida, engine='xlsxwriter') as writer:
    for nome, df in planilhas.items():
        df.to_excel(writer, sheet_name=nome, index=False)

print(f"✅ Planilha 'TopicosCorpusTodo' adicionada ao arquivo:\n{saida}")


✅ Planilha 'TopicosCorpusTodo' adicionada ao arquivo:
/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado_topicos.xlsx


In [22]:
import pandas as pd

# Caminhos
entrada = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado_topicos.xlsx'
saida = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado_topicos_long.xlsx'

# Carrega todas as planilhas
planilhas = pd.read_excel(entrada, sheet_name=None)

# Seleciona a planilha TopicosCorpusTodo
df = planilhas["TopicosCorpusTodo"]

# Define os campos que queremos transformar (sem Ano e BM25)
colunas_gerais = [
    'Document', 'Topic', 'Name', 'Representation',
    'Representative_Docs', 'Top_n_words', 'Probability',
    'Representative_document'
]

# Cria dois DataFrames com os nomes padronizados + tipo
def extrair_tipo(df, prefixo, tipo):
    colunas_prefixed = [f"{prefixo}_{col}" for col in colunas_gerais]
    df_temp = df[["CodigoPronunciamento"] + colunas_prefixed].copy()
    df_temp.columns = ["CodigoPronunciamento"] + colunas_gerais  # renomeia para os nomes base
    df_temp["Tipo"] = tipo
    return df_temp

df_norm = extrair_tipo(df, "NormConclusao", "NormConclusao")
df_aval = extrair_tipo(df, "AvalConclusao", "AvalConclusao")

# Junta os dois DataFrames
df_long = pd.concat([df_norm, df_aval], ignore_index=True)

# Renomeia para alinhar ao formato do AnaliseAnual
df_long = df_long.rename(columns={
    "Document": "Trecho",
    "Name": "Topico"
})

# Ordena
df_long = df_long.sort_values(["CodigoPronunciamento", "Tipo", "Topic"])

# Adiciona como nova aba ao Excel
planilhas["TopicosCorpusTodo_Long"] = df_long

# Salva o novo arquivo com todas as planilhas
with pd.ExcelWriter(saida, engine='xlsxwriter') as writer:
    for nome, df_sheet in planilhas.items():
        df_sheet.to_excel(writer, sheet_name=nome, index=False)

print(f"✅ Arquivo salvo com nova planilha 'TopicosCorpusTodo_Long' no formato correto!\n📁 {saida}")


✅ Arquivo salvo com nova planilha 'TopicosCorpusTodo_Long' no formato correto!
📁 /content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado_topicos_long.xlsx


In [24]:
import pandas as pd
import os
from glob import glob

# Caminhos
entrada = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado_topicos_long.xlsx'
saida = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado_with_csv_values.xlsx'
base_dir = '/content/drive/MyDrive/falando_nela_v2/data/topicos_gerados/v_12'

# Carrega todas as planilhas
planilhas = pd.read_excel(entrada, sheet_name=None)
df_analise = planilhas["AnaliseAnual"].copy()

# Cria dicionário para armazenar dados por tipo
substituicoes_dict = {
    "NormConclusao": [],
    "AvalConclusao": []
}

# Processa arquivos CSV
arquivos_csv = glob(os.path.join(base_dir, "*_*_topicos_complementacao.csv"))

for caminho in arquivos_csv:
    nome = os.path.basename(caminho)

    if "NormConclusao" in nome:
        tipo = "NormConclusao"
    elif "AvalConclusao" in nome:
        tipo = "AvalConclusao"
    else:
        continue  # ignora outros arquivos

    df_csv = pd.read_csv(caminho)

    if "CodigoPronunciamento" not in df_csv.columns or tipo not in df_csv.columns:
        continue

    substituicoes_dict[tipo].append(df_csv[["CodigoPronunciamento", tipo]])

# Concatena os valores encontrados
df_norm = pd.concat(substituicoes_dict["NormConclusao"], ignore_index=True) if substituicoes_dict["NormConclusao"] else pd.DataFrame(columns=["CodigoPronunciamento", "NormConclusao"])
df_aval = pd.concat(substituicoes_dict["AvalConclusao"], ignore_index=True) if substituicoes_dict["AvalConclusao"] else pd.DataFrame(columns=["CodigoPronunciamento", "AvalConclusao"])

# Junta os dois em um só DataFrame por CodigoPronunciamento
df_substituto = pd.merge(df_norm, df_aval, on="CodigoPronunciamento", how="outer")

# Atualiza os valores na planilha AnaliseAnual, mantendo todas as colunas
df_analise.set_index("CodigoPronunciamento", inplace=True)
df_substituto.set_index("CodigoPronunciamento", inplace=True)

for col in ["NormConclusao", "AvalConclusao"]:
    if col in df_substituto.columns:
        df_analise.update(df_substituto[[col]])

df_analise.reset_index(inplace=True)

# Substitui a planilha no dicionário
planilhas["AnaliseAnual"] = df_analise

# Salva novo Excel com todas as planilhas
with pd.ExcelWriter(saida, engine='xlsxwriter') as writer:
    for nome, df_plan in planilhas.items():
        df_plan.to_excel(writer, sheet_name=nome[:31], index=False)

print(f"✅ Planilha 'AnaliseAnual' atualizada com dados dos CSVs!\n📁 Arquivo salvo em:\n{saida}")


✅ Planilha 'AnaliseAnual' atualizada com dados dos CSVs!
📁 Arquivo salvo em:
/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado_with_csv_values.xlsx


In [25]:
import pandas as pd

# Caminhos
entrada = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_reorganizado_with_csv_values.xlsx'
saida = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_final_padronizado.xlsx'

# Carrega todas as planilhas
planilhas = pd.read_excel(entrada, sheet_name=None)

# Remove a planilha TopicosCorpusTodo, se existir
planilhas.pop("TopicosCorpusTodo", None)

# Renomeia TopicosCorpusTodo_Long → TopicosCorpusTodo
if "TopicosCorpusTodo_Long" in planilhas:
    planilhas["TopicosCorpusTodo"] = planilhas.pop("TopicosCorpusTodo_Long")

# Reorganiza cada planilha: CodigoPronunciamento como primeira coluna
for nome, df in planilhas.items():
    if "CodigoPronunciamento" in df.columns:
        colunas = ["CodigoPronunciamento"] + [col for col in df.columns if col != "CodigoPronunciamento"]
        planilhas[nome] = df[colunas]

# Salva o novo arquivo padronizado
with pd.ExcelWriter(saida, engine='xlsxwriter') as writer:
    for nome, df_plan in planilhas.items():
        df_plan.to_excel(writer, sheet_name=nome[:31], index=False)

print(f"✅ Novo arquivo salvo com planilha renomeada, reorganizada e padronizada:\n📁 {saida}")


✅ Novo arquivo salvo com planilha renomeada, reorganizada e padronizada:
📁 /content/drive/MyDrive/falando_nela_v2/data/xlsx_final_padronizado.xlsx


In [26]:
# prompt: Renomeie /content/drive/MyDrive/falando_nela_v2/data/xlsx_final_padronizado.xlsx para dados_para_analise.xlsx

import os

# Caminhos
caminho_antigo = '/content/drive/MyDrive/falando_nela_v2/data/xlsx_final_padronizado.xlsx'
caminho_novo = '/content/drive/MyDrive/falando_nela_v2/data/dados_para_analise.xlsx'

# Renomeia o arquivo
try:
    os.rename(caminho_antigo, caminho_novo)
    print(f"Arquivo renomeado com sucesso para: {caminho_novo}")
except FileNotFoundError:
    print(f"Erro: Arquivo não encontrado em: {caminho_antigo}")
except OSError as e:
    print(f"Erro ao renomear arquivo: {e}")


Arquivo renomeado com sucesso para: /content/drive/MyDrive/falando_nela_v2/data/dados_para_analise.xlsx


Limpar SQLITE[texto do link](https://)

In [27]:
import sqlite3
import pandas as pd

# Caminho do banco no Google Drive
caminho_db = '/content/drive/MyDrive/falando_nela_v2/data/DiscursosSenadores.sqlite'

# Conecta ao banco
con = sqlite3.connect(caminho_db)

# Lista as tabelas
tabelas = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", con)
print("📋 Tabelas encontradas:")
display(tabelas)

# Exibe o schema de cada tabela
for nome in tabelas['name']:
    print(f"\n🔍 Schema da tabela: {nome}")
    schema = pd.read_sql_query(f"PRAGMA table_info('{nome}');", con)
    display(schema)



📋 Tabelas encontradas:


,name
0,Senadores
1,sqlite_sequence
2,SenadoresCargos
3,SenadoresHistoricoAcademico
4,DiscursosAparteantes
5,DiscursosPublicacoes
6,DiscursosSessoes
7,SenadoresLiderancas
8,SenadoresComissoes
9,SenadoresFiliacoes



🔍 Schema da tabela: Senadores


,cid,name,type,notnull,dflt_value,pk
0,0,CodigoParlamentar,INTEGER,0,None,1
1,1,NomeParlamentar,TEXT,1,None,0
2,2,NomeCompletoParlamentar,TEXT,0,None,0
3,3,SexoParlamentar,TEXT,0,None,0
4,4,SiglaPartidoParlamentar,TEXT,0,None,0
5,5,UfParlamentar,TEXT,0,None,0
6,6,UrlFotoParlamentar,TEXT,0,None,0
7,7,EmailParlamentar,TEXT,0,None,0
8,8,NomeProfissao,TEXT,0,None,0
9,9,IndicadorAtividadePrincipal,TEXT,0,None,0



🔍 Schema da tabela: sqlite_sequence


...

,cid,name,type,notnull,dflt_value,pk
0,0,CodigoPronunciamento,INTEGER,0,None,1
1,1,Similaridade_base_democracia,REAL,0,None,0
2,2,Similaridade_ultrapassada,REAL,0,None,0
3,3,Similaridade_prejudica_economia,REAL,0,None,0
4,4,Similaridade_direitos_demais,REAL,0,None,0
5,5,Similaridade_nova_constituinte,REAL,0,None,0
6,6,Similaridade_ffaa_poder_moderador,REAL,0,None,0
7,7,Similaridade_voltar_ditadura,REAL,0,None,0
8,8,Similaridade_governo_nao_respeita,REAL,0,None,0
9,9,Similaridade_camara_nao_respeita,REAL,0,None,0



🔍 Schema da tabela: DiscursosNarrativas


,cid,name,type,notnull,dflt_value,pk
0,0,CodigoPronunciamento,INTEGER,0,None,1
1,1,MencionaConstituicao,BOOLEAN,0,None,0
2,2,NormPredicacao,TEXT,0,None,0
3,3,NormImplicacao,TEXT,0,None,0
4,4,NormConclusao,TEXT,0,None,0
5,5,NormTrecho,TEXT,0,None,0
6,6,AvalPredicacao,TEXT,0,None,0
7,7,AvalImplicacao,TEXT,0,None,0
8,8,AvalConclusao,TEXT,0,None,0
9,9,AvalTrecho,TEXT,0,None,0



🔍 Schema da tabela: DiscNarrativasTopicos


,cid,name,type,notnull,dflt_value,pk
0,0,CodigoPronunciamento,INTEGER,0,None,1
1,1,NormPredicacao_topicos,REAL,0,None,0
2,2,NormImplicacao_topicos,REAL,0,None,0
3,3,NormConclusao_topicos,REAL,0,None,0
4,4,AvalPredicacao_topicos,REAL,0,None,0
5,5,AvalImplicacao_topicos,REAL,0,None,0
6,6,AvalConclusao_topicos,REAL,0,None,0


In [28]:
import sqlite3
import shutil

# Caminhos
caminho_original = '/content/drive/MyDrive/falando_nela_v2/data/DiscursosSenadores.sqlite'
caminho_novo = '/content/drive/MyDrive/falando_nela_v2/data/DiscursosSenadores_limpo.sqlite'

# Copia o banco original
shutil.copyfile(caminho_original, caminho_novo)

# Conecta ao banco copiado
con = sqlite3.connect(caminho_novo)
cursor = con.cursor()

# Tabelas a remover completamente
tabelas_para_deletar = [
    'DiscursosAnalises',
    'DiscursosPesquisa',
    'DiscursosNarrativas',
    'DiscNarrativasTopicos'
]

# DROP TABLE para cada uma
for tabela in tabelas_para_deletar:
    print(f"🗑️ Removendo tabela: {tabela}")
    cursor.execute(f'DROP TABLE IF EXISTS {tabela};')

# Compacta o banco
print("🧼 Compactando banco com VACUUM...")
cursor.execute("VACUUM;")

# Salva e fecha
con.commit()
con.close()

print(f"✅ Banco salvo sem as tabelas deletadas:\n📁 {caminho_novo}")


🗑️ Removendo tabela: DiscursosAnalises
🗑️ Removendo tabela: DiscursosPesquisa
🗑️ Removendo tabela: DiscursosNarrativas
🗑️ Removendo tabela: DiscNarrativasTopicos
🧼 Compactando banco com VACUUM...
✅ Banco salvo sem as tabelas deletadas:
📁 /content/drive/MyDrive/falando_nela_v2/data/DiscursosSenadores_limpo.sqlite
